# Calculate NPV

## Setup

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from pathlib import Path
import json

# Plottting libraries
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")
%matplotlib inline
%config InlineBackend.figure_format ='retina'

<IPython.core.display.Javascript object>

## Configure

In [3]:
# Set data path
file_path = Path("./data")
finance_data_path = Path("./data/finance_data")

# File names
master_file = "master_data.xlsx"
cashflow_file = "CashFlow.xlsx"
ratios_file = "Ratios.xlsx"
price_file = "price.csv"
npv_file = "npv.csv"

<IPython.core.display.Javascript object>

## Functions

In [4]:
# Load the raw cashflow data and convert to dataframe
def cashflow_loader(file_path, ticker):
    # Open the file
    df = pd.read_excel(file_path, sheet_name=ticker, engine="openpyxl", skiprows=1)
    # Set index column,
    df.set_index("Cash Flow Statement", inplace=True)
    # Drop LTM if it exists and set the index to years
    df.drop("LTM", axis=1, errors="ignore", inplace=True)
    # Set the index to years
    df.columns = pd.to_datetime(df.columns).year
    df = df.T
    # Rename column
    df = df.loc[:, ["Cash Flow per Share"]]
    df.columns = ["FCF_share"]

    return df

<IPython.core.display.Javascript object>

In [24]:
def growth_calc(
    starting_fcf, discount_rate, growth_rate1, growth_rate2, perpetual_rate
):
    # Set a range for first 5 years
    years = np.arange(1, 6)

    # Reset rates as fraction from percentage
    discount_rate /= 100
    growth_rate1 /= 100
    growth_rate2 /= 100
    perpetual_rate /= 100

    # Calculate FCF for first stage
    fcf_stage1 = starting_fcf * (1 + growth_rate1) ** years
    # Calculate FCF for second stage
    fcf_stage2 = fcf_stage1[-1] * (1 + growth_rate2) ** (years)
    # Set to dataframes
    df1 = pd.DataFrame(data=fcf_stage1, index=years, columns=["value"])
    df2 = pd.DataFrame(data=fcf_stage2, index=years + 5, columns=["value"])
    # Add a terminal value
    terminal_value = (
        fcf_stage2[-1] * (1 + perpetual_rate) / (discount_rate - perpetual_rate)
    )
    # Concatenate the dataframes and set last row as terminal value
    df = pd.concat([df1, df2])
    # Reset index to create a years column
    df = df.reset_index().rename(columns={"index": "years"})
    df.loc[11] = 10, terminal_value

    return df

<IPython.core.display.Javascript object>

## Calculate NPV

In [56]:
def pv_calc(data, discount_rate):
    df = data.copy()

    # Reset rate as fraction from percentage
    discount_rate /= 100
    # Calculate present value
    df["PV"] = df["value"] * (1 + discount_rate) ** (
        df.loc[df["years"] == 1, "years"].values - df["years"].values - 1
    )
    
    return df

<IPython.core.display.Javascript object>

In [ ]:
def npv_calc(starting_fcf, discount_rate, growth_rate1, growth_rate2, perpetual_rate):
    df_fcf = growth_calc(starting_fcf, discount_rate, growth_rate1, growth_rate2, perpetual_rate)
    df_pv = pv_calc(df_fcf, discount_rate)
    npv = df_pv['PV'].sum()
    
    return npv, df_pv

In [ ]:
# Load dataframe of all input values
df_params = pd.read_csv(somefile.csv)

In [58]:
npv_dict = {}

for name in company_names:
    npv_p10 = npv_calc(starting_fcf, discount_rate, growth_rate1, growth_rate2, perpetual_rate)
    npv_p50 = npv_calc(starting_fcf, discount_rate, growth_rate1, growth_rate2, perpetual_rate)
    npv_p90 = npv_calc(starting_fcf, discount_rate, growth_rate1, growth_rate2, perpetual_rate)
    
    risked_npv = 0.3 * npv_p10 + 0.4 * npv_p50 + 0.3 * npv_p90
    
    npv_dict[name] = {'npv':risked_npv, 'npv_p10':npv_p10, 'npv_p50':npv_p50, 'npv_p90':npv_p90, 'pv_data':df_pv}

SyntaxError: invalid syntax (<ipython-input-58-14e4333dee09>, line 4)

ERROR:root:Cannot parse: 4:14:     npv_p10 = 
Traceback (most recent call last):
  File "/Users/telamon/miniconda3/envs/invest/lib/python3.7/site-packages/lab_black.py", line 218, in format_cell
    formatted_code = _format_code(cell)
  File "/Users/telamon/miniconda3/envs/invest/lib/python3.7/site-packages/lab_black.py", line 29, in _format_code
    return format_str(src_contents=code, mode=FileMode())
  File "/Users/telamon/miniconda3/envs/invest/lib/python3.7/site-packages/black/__init__.py", line 1077, in format_str
    src_node = lib2to3_parse(src_contents.lstrip(), mode.target_versions)
  File "/Users/telamon/miniconda3/envs/invest/lib/python3.7/site-packages/black/parsing.py", line 132, in lib2to3_parse
    raise exc from None
black.parsing.InvalidInput: Cannot parse: 4:14:     npv_p10 = 


### Build Company FCF

In [10]:
# Load tickers
df_tickers = pd.read_excel(
    file_path / master_file, sheet_name="Names", engine="openpyxl"
)

<IPython.core.display.Javascript object>

In [11]:
df_tickers = df_tickers.iloc[:1]

<IPython.core.display.Javascript object>

In [12]:
company_fcf = {}
mask = df_tickers["Type"] == "Regular"
for name in df_tickers.loc[mask, "Ticker"]:
    company_fcf[name] = cashflow_loader(finance_data_path / cashflow_file, name)

<IPython.core.display.Javascript object>

### FCF Starting Point Values

In [13]:
fcf_dict = {}
for key, value in company_fcf.items():
    fcf_dict[key] = value.loc[value.last_valid_index(), "FCF_share"]

df_start = pd.DataFrame.from_dict(
    data=fcf_dict, orient="index", columns=["start_value"]
)

<IPython.core.display.Javascript object>

In [14]:
df_start

,start_value
ACN,13.23


<IPython.core.display.Javascript object>

### NPV Calculations

## Export Data

In [ ]:
# df.to_csv(file_path/forecasts_file)